# Router Engine

## Setup

In [3]:
from dotenv import load_dotenv
import os

load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')


In [4]:
import nest_asyncio

nest_asyncio.apply()

## Load Data

To download this paper, below is the needed code:

#!wget "https://openreview.net/pdf?id=VtmBAGCN7o" -O metagpt.pdf

In [5]:
from llama_index.core import SimpleDirectoryReader

# load documents
documents = SimpleDirectoryReader(input_files=["metagpt.pdf"]).load_data()

## Define LLM and Embedding model

In [6]:
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

In [7]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

Settings.llm = OpenAI(model="gpt-3.5-turbo")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002")

## Define Summary Index and Vector Index over the Same Data

In [8]:
from llama_index.core import SummaryIndex, VectorStoreIndex

summary_index = SummaryIndex(nodes)
vector_index = VectorStoreIndex(nodes)

## Define Query Engines and Set Metadata

In [9]:
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
vector_query_engine = vector_index.as_query_engine()

In [10]:
from llama_index.core.tools import QueryEngineTool


summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description=(
        "Useful for summarization questions related to MetaGPT"
    ),
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=(
        "Useful for retrieving specific context from the MetaGPT paper."
    ),
)

## Define Router Query Engine

In [11]:
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector


query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,
        vector_tool,
    ],
    verbose=True
)

In [12]:
response = query_engine.query("What is the summary of the document?")
print(str(response))

Selecting query engine 0: Useful for summarization questions related to MetaGPT.
The document introduces MetaGPT, a meta-programming framework that utilizes Standard Operating Procedures (SOPs) to enhance the problem-solving capabilities of multi-agent systems based on Large Language Models (LLMs). MetaGPT models a group of agents as a simulated software company, emphasizing role specialization, workflow management, and efficient sharing mechanisms. It incorporates an executable feedback mechanism to improve code generation quality during runtime and has shown state-of-the-art performance in various benchmarks. The work also inspires future research on human-inspired techniques for artificial multi-agent systems and aims to regulate LLM-based multi-agent frameworks. The document discusses the development process using MetaGPT for software development, starting with user input, collaboration with a professional team, and detailed documentation for system design. Additionally, it provide

In [13]:
print(len(response.source_nodes))

34


In [15]:
response = query_engine.query(
    "How do agents share information with other agents?"
)
print(str(response))

Selecting query engine 0: MetaGPT is likely to provide useful information for summarization questions related to how agents share information with other agents..
Agents share information with other agents by utilizing a shared message pool where they publish structured messages and access messages from other entities. Additionally, agents can subscribe to specific information based on their profiles, allowing them to extract relevant information efficiently. This method of sharing information eliminates the need for one-to-one communication and enhances communication efficiency within the multi-agent system.


## Let's put everything together

In [ ]:
from utils import get_router_query_engine

query_engine = get_router_query_engine("metagpt.pdf")

In [ ]:
response = query_engine.query("Tell me about the ablation study results?")
print(str(response))